<a href="https://colab.research.google.com/github/TI1307/DM_Project/blob/main/notebooks/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing and cleaning

# 4.1 Data Quality **Issues**


In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
########    LIBRARY INVENTORY TABLE      ###############
LIBRARY_INVENTORY= pd.read_excel("/content/drive/MyDrive/Book Recomendation/data/LIBRARY INVENTORY.xlsx")
print ('Number of instance in the LIBRARY INVENTORY' , (LIBRARY_INVENTORY.shape[0]))
print ('Number of attributes in the LIBRARY INVENTORY' , (LIBRARY_INVENTORY.shape[1]))
print(LIBRARY_INVENTORY.head(3))
print ('-'*20)
########  LIBRARY_CARALOGUE TABLE     ###############
LIBRARY_CATALOGUE= pd.read_excel("/content/drive/MyDrive/Book Recomendation/data/library catalogue.xlsx")
print ('Number of instance in the LIBRARY CARALOGUE' , (LIBRARY_CATALOGUE.shape[0]))
print ('Number of attributes in the LIBRARY INVENTORY' , (LIBRARY_CATALOGUE.shape[1]))
print ('-'*20)
#######    BORROWING TABLE             ###########################
BORROWINGS= pd.read_excel("/content/drive/MyDrive/Book Recomendation/data/Borrowings.xlsx")
print ('Number of instance in the BORROWINGS TABLE ' , (BORROWINGS.shape[0]))
print ('Number of attributes in the BORROWINGS TABLE ' , (BORROWINGS.shape[1]))
print(BORROWINGS.head(3))

Mounted at /content/drive
Number of instance in the LIBRARY INVENTORY 4255
Number of attributes in the LIBRARY INVENTORY 16
  N° inventaire Date de réception  \
0         62504               NaT   
1        111442               NaT   
2             1        2021-12-19   

                                               Titre            Auteur  \
0  Programmation mathématique : théorie et algori...    Minoux, Michel   
1  Calcul des probabilités : cours, exercices et ...  Foata, Dominique   
2            Calcul DIfferentiel et Integral Tome 01       Piskounov,N   

                                  Editeur  Date d'édition Lieu d'édition  \
0                               Lavoisier          2008.0          Paris   
1                                   Dunod          2012.0          Paris   
2  Office des Publications Universitaires          2006.0        algerie   

  Edition Mode d'acquisition        Statut Fournisseur             N° achat  \
0  2e éd.             Passif           NaN    

# **4.1.1 Missing Values**

In [5]:
# gives a summery of missing values
def missing_values_summary(df):
  missing_count = df.isnull().sum()
  missing_percent = (missing_count/ len(df))*100

  summery = pd.DataFrame({
      'missing_count': missing_count,
      'missing_percent': missing_percent
  })
  summery = summery.sort_values('missing_percent', ascending=False)
  return summery
import pandas as pd

def handle_missing_values(df, strategy_dict):
    """
    Fill missing values in a DataFrame according to a strategy dictionary.

    Parameters:
        df (pd.DataFrame): Input dataframe
        strategy_dict (dict): {column_name: strategy}, where strategy can be:
            - 'mean'   → fill numeric with mean
            - 'median' → fill numeric with median
            - 'mode'   → fill categorical with mode
            - value    → fill with this value directly

    Returns:
        pd.DataFrame: DataFrame with missing values handled
    """
    for col, strategy in strategy_dict.items():
        if strategy == 'mean':
            df[col] = df[col].fillna(df[col].mean())
        elif strategy == 'median':
            df[col] = df[col].fillna(df[col].median())
        elif strategy == 'mode':
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            # Use the provided value
            df[col] = df[col].fillna(strategy)
    return df




# **4.1.2 Outliers**

# **4.1.3 Duplicate Data**

In [6]:
def checkDuplicat (df, table_name,  keep=False):
    dup=df.duplicated()
    dup_count=dup.sum()
    print(f'Number of duplicate rows in {table_name} table is = %d' % (dup_count))
    if dup_count > 0:
      print (f'\n Duplicare row in the table {table_name}:')
      dup_rows=df[dup]
      for idx in dup_rows.index:
          print(f'\n row number :{idx}')
          print(df.loc[idx])
          print('-' * 50)




# **4.2 Aggregation**

# **4.3 Sampling**

# **4.4 Discretization**

# **Borowing Table** #

In [8]:
# Cleaningg the borrowing table
# table name : BORROWINGS

print ('Number of instance in the BORROWINGS TABLE ' , (BORROWINGS.shape[0]))
print ('Number of attributes in the BORROWINGS TABLE ' , (BORROWINGS.shape[1]))

# summery about missing values :
print(missing_values_summary(BORROWINGS))

#check duplicate :
checkDuplicat(BORROWINGS , "BORROWINGS TABLE ")

Number of instance in the BORROWINGS TABLE  422
Number of attributes in the BORROWINGS TABLE  11
                  missing_count  missing_percent
Date réservation            414        98.104265
Cote                         14         3.317536
Numéro                        0         0.000000
Titre                         0         0.000000
Type de document              0         0.000000
Nom                           0         0.000000
N° lecteur                    0         0.000000
Prénom                        0         0.000000
Catégorie                     0         0.000000
Date prêt                     0         0.000000
Date retour                   0         0.000000
Number of duplicate rows in BORROWINGS TABLE  table is = 0


# **LIBRARY INVENTORY**

In [9]:
# Cleaningg the LIBRARY INVENTORY table
# table name : LIBRARY_INVENTORY

print ('Number of instance in the LIBRARY_INVENTORY TABLE ' , (LIBRARY_INVENTORY.shape[0]))
print ('Number of attributes in the LIBRARY_INVENTORY TABLE ' , (LIBRARY_INVENTORY.shape[1]))

# summery about missing values :
print(missing_values_summary(LIBRARY_INVENTORY))

#check duplicate :
checkDuplicat(LIBRARY_INVENTORY, "LIBRARY_INVENTORY")

Number of instance in the LIBRARY_INVENTORY TABLE  4255
Number of attributes in the LIBRARY_INVENTORY TABLE  16
                    missing_count  missing_percent
Observation                  4250        99.882491
Prix                         3397        79.835488
Fournisseur                  3353        78.801410
N° achat                     3353        78.801410
Date de réception            3227        75.840188
Edition                      3179        74.712103
Editeur                      1204        28.296122
Lieu d'édition                429        10.082256
Date d'édition                220         5.170388
Auteur                        105         2.467685
Cote                           83         1.950646
Localisation                   46         1.081081
Statut                         40         0.940071
Titre                           0         0.000000
N° inventaire                   0         0.000000
Mode d'acquisition              0         0.000000
Number of duplicate r

# **LIBRARY_CATALOGUE** #

In [10]:
# Cleaningg the LIBRARY_CATALOGUE table
# table name : LIBRARY_CATALOGUE

print ('Number of instance in the LIBRARY_CATALOGUE TABLE ' , (LIBRARY_CATALOGUE.shape[0]))
print ('Number of attributes in the LIBRARY_CATALOGUE TABLE ' , (LIBRARY_CATALOGUE.shape[1]))

# summery about missing values :
print(missing_values_summary(LIBRARY_CATALOGUE))

#check duplicate :
checkDuplicat(LIBRARY_CATALOGUE, "LIBRARY_CATALOGUE")

Number of instance in the LIBRARY_CATALOGUE TABLE  620
Number of attributes in the LIBRARY_CATALOGUE TABLE  5
               missing_count  missing_percent
Nbr. Exp.                104        16.774194
ISBN, ISSN...             42         6.774194
Cote                      32         5.161290
Auteur                    17         2.741935
Titre                      0         0.000000
Number of duplicate rows in LIBRARY_CATALOGUE table is = 0
